# **Análise de Dados COVID-19**

## 1\. Introdução

### **1.1. TLDR**

 - **Dashboard**:
  - Google Data Studio ([link](https://lookerstudio.google.com/reporting/fd8ffb38-fc58-4cba-be56-3d051f15dccc)).
 - **Processamento**:
  - Kaggle Notebook ([link](https://www.kaggle.com/code/theu011/an-lise-de-dados-covid-19)).
 - **Fontes**:
  - Casos pela universidade John Hopkins ([link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports));
  - Vacinação pela universidade de Oxford ([link](https://covid.ourworldindata.org/data/owid-covid-data.csv)).

### **1.2. Pandemia Coronavírus 2019**

> A COVID-19 é uma infecção respiratória aguda causada pelo coronavírus SARS-CoV-2, potencialmente grave, de elevada transmissibilidade e de distribuição global, tornou-se um dos maiores desafios globais das últimas décadas, afetando milhões de vidas em todo o mundo. Desde seu surgimento no final de 2019, a disseminação do vírus gerou impactos significativos em áreas como saúde pública, economia e políticas governamentais.

### **1.3\. Contexto**
Nesta análise exploratória de dados (EDA), buscamos compreender os principais padrões e tendências da pandemia de COVID-19, explorando um conjunto de dados históricos que abrange casos confirmados e mortes.  
Esses dados foram coletados e organizados de maneira a permitir uma segmentação detalhada por regiões geográficas, períodos de tempo e outras dimensões relevantes.

Utilizando técnicas avançadas de visualização de dados e estatísticas descritivas, investigamos a evolução temporal da pandemia, identificando marcos significativos como picos de infecção e períodos de declínio.  
Além disso, a análise permitirá destacar o impacto do vírus em diferentes regiões, revelando quais áreas foram mais severamente atingidas.

---

## 2\. Análise Exploratória de Dados


#### **Pacotes e bibliotecas**

In [142]:
from datetime import datetime, timedelta

import pandas as pd
import numpy as np

In [143]:
def date_range(start_date: datetime, end_date: datetime):

    date = start_date
    while date < end_date:
        yield date
        date += timedelta(days=1)


def get_trend(rate: float):
  
  if np.isnan(rate):
    return np.nan

  if rate < 0.85:
    status = 'downward'
  elif rate > 1.15:
    status = 'upward'
  else:
    status = 'stable'

  return status

### 2.1. Casos Covid

Os dados sobre **casos da COVID-19** são compilados pelo centro de ciência de sistemas e engenharia da universidade americana **John Hopkins** ([link](https://www.jhu.edu)). Os dados são atualizados diariamente deste janeiro de 2020 com uma granularidade temporal de dias e geográfica de regiões de países (estados, condados, etc.). O website do projeto pode ser acessado neste [link](https://systems.jhu.edu/research/public-health/ncov/) enquanto os dados, neste [link](https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports). Abaixo estão descritos os dados derivados do seu processamento.

 - **date**: data de referência;
 - **state**: estado;
 - **country**: país;
 - **population**: população estimada;
 - **confirmed**: número acumulado de infectados;
 - **confirmed_per_day**: número diário de infectados;
 - **confirmed_moving_avg_7day**: média móvel de 7 dias do número diário de infectados;
 - **confirmed_moving_avg_7day_rate_14day**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **deaths**: número acumulado de mortos;
 - **deaths_per_day**: número diário de mortos;
 - **deaths_moving_avg_7day**: média móvel de 7 dias do número diário de mortos;
 - **deaths_moving_avg_7day_rate_14day**: média móvel de 7 dias dividido pela média móvel de 7 dias de 14 dias atrás;
 - **day**: dia de referência;
 - **month**: mês de referência.

#### **2.1.1\. Extração**

Define a data inicial e final da coleta de dados.

In [144]:
start_date = datetime(2021,  1,  1)
end_date   = datetime(2021, 12, 31)

Utiliza a função **`date_range`** em um laço **`for`** para iterar sobre o intervalo de datas especificado (`start_date` e `end_date`).  

Altera a string da **URL** dos dados, montando o caminho para o arquivo CSV diário correspondente a cada data.  

Carrega o arquivo CSV em um DataFrame chamado **`case`**.  

Dropa as colunas que não têm interesse com o método **`drop`**, mantendo apenas as informações relevantes.  

Em **`case.query`**, seleciona os países de interesse.

Define a coluna **`Date`** para o tipo **`datetime`** e formata com a data correspondente ao arquivo atual.  

Verifica se **`cases_is_empty`** é `True`:  
   - Se for, atribui o DataFrame **`case`** diretamente a **`cases`** e marca **`cases_is_empty`** como `False`.  
   - Caso contrário, concatena o DataFrame atual (**`case`**) ao DataFrame consolidado (**`cases`**) com o método **`pd.concat`**.  

O resultado final será um DataFrame chamado **`cases`**, contendo dados combinados de todos os dias do intervalo, com informações filtradas e organizadas.  


In [145]:
cases = None
cases_is_empty = True

for date in date_range(start_date=start_date, end_date=end_date):

    date_str = date.strftime('%m-%d-%Y')
    data_source_url = f'https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/{date_str}.csv'

    case = pd.read_csv(data_source_url, sep=',')
    case = case.drop(['FIPS', 'Admin2', 'Last_Update', 'Lat', 'Long_', 'Recovered', 'Active', 'Combined_Key', 'Case_Fatality_Ratio'], axis=1)
    case = case.query('Country_Region in ["Brazil", "Japan", "China", "Russia", "France", "Italy", "Ukraine", "Spain", "Canada", "Mexico", "Peru", "Colombia"]').reset_index(drop=True)
    case['Date'] = pd.to_datetime(date.strftime('%Y-%m-%d'))

    if cases_is_empty:
        cases = case
        cases_is_empty = False
    else:
        cases = pd.concat([cases, case], axis=0, ignore_index=True)

Salva o DataFrame consolidado **`cases`** em um arquivo CSV chamado `cases_covid.csv`.  
Redefine a variável **`cases`** como `None`, liberando memória e sinalizando que o DataFrame não está mais em uso.


In [146]:
cases.to_csv('cases_covid.csv', sep=',', index=False)
cases = None

Carrega o arquivo CSV `cases_covid.csv` em um DataFrame chamado **`cases_covid`**.  
O parâmetro `parse_dates=[-1]` converte automaticamente a última coluna do arquivo em formato de data (`datetime`).  


In [147]:
cases_covid = pd.read_csv('cases_covid.csv', sep=',', parse_dates=[-1])

#### **2.1.2\. Transformação**

Renomeia as colunas no DataFrame

In [148]:
cases_covid =  cases_covid.rename(columns={
    'Province_State': 'state',
    'Country_Region': 'country'
})

O método **`rename`** é utilizado com uma função **lambda** para transformar todos os nomes de colunas do DataFrame **`cases_covid`** em letras minúsculas.  

In [149]:
cases_covid = cases_covid.rename(columns=lambda col: col.lower())

Define um dicionário que contém mapeamentos de nomes de estados brasileiros sem acentos para suas respectivas versões acentuadas.   
A coluna **`state`** do DataFrame **`cases_covid`** é atualizada utilizando o método **`apply`** com uma função **lambda**.  

In [150]:
states_map = {
    'Amapa': 'Amapá',
    'Ceara': 'Ceará',
    'Espirito Santo': 'Espírito Santo',
    'Goias': 'Goiás',
    'Para': 'Pará',
    'Paraiba': 'Paraíba',
    'Parana': 'Paraná',
    'Piaui': 'Piauí',
    'Rondonia': 'Rondônia',
    'Sao Paulo': 'São Paulo'
}

cases_covid['state'] = cases_covid['state'].apply(lambda state: states_map.get(state) if state in states_map.keys() else state)

Cria uma nova coluna chamada **`day`** e outra chamada **`month`** no DataFrame **`cases_covid`**.  
O atributo **`.dt.day`** extrai o número correspondente ao dia do mês (1 a 31).  
O atributo **`.dt.month`** extrai o número correspondente ao mês (1 para janeiro, 2 para fevereiro, etc.).  


In [151]:
cases_covid['day'] = pd.to_datetime(cases_covid['date']).dt.day
cases_covid['month'] = pd.to_datetime(cases_covid['date']).dt.month

Aplica o método **`ffill`** ao DataFrame **`cases_covid`**, preenchendo os valores ausentes (`NaN`) de cada coluna com o último valor válido encontrado acima na mesma coluna.  
Caso o primeiro valor seja ausente, ele permanecerá como `NaN`, já que não há valores acima para preencher.

In [152]:
cases_covid = cases_covid.ffill()

Cria uma nova coluna chamada **`population`** no DataFrame **`cases_covid`**. Adicionar uma estimativa da população com base nos dados disponíveis.  
Remove a coluna **`incident_rate`**, pois ela já foi utilizada para calcular a população e não é mais necessária para análises subsequentes.

In [153]:
cases_covid['population'] = round(100000 * (cases_covid['confirmed'] / cases_covid['incident_rate']))
cases_covid = cases_covid.drop('incident_rate', axis=1)

**`cases = None`** e **`cases_is_empty = True`** são usados para controlar o início do DataFrame e quando concatenar os resultados.

O loop percorre todos os estados únicos em **`cases_covid['state']`**.

Para cada estado, os dados são filtrados e ordenados. Calculam-se as métricas diárias e médias móveis para casos confirmados e óbitos, incluindo tendências com a função **`get_trend`**.

Os dados processados de cada estado são concatenados ao DataFrame **`cases`**.

O DataFrame consolidado é atribuído a **`cases_covid`**, e **`cases`** é redefinido como **`None`** para liberar memória.

In [154]:
cases = None
cases_is_empty = True

for state in cases_covid['state'].drop_duplicates():
    cases_per_state = cases_covid.query(f'state == "{state}"').reset_index(drop=True)
    cases_per_state = cases_per_state.sort_values(by=['date'])

    cases_per_state['confirmed_per_day'] = cases_per_state['confirmed'].diff(periods=1)
    cases_per_state['confirmed_per_day'] = cases_per_state['confirmed_per_day'].clip(lower=0)
    cases_per_state['confirmed_moving_avg_7day'] = np.ceil(cases_per_state['confirmed_per_day'].rolling(window=7).mean())
    cases_per_state['confirmed_moving_avg_7day_rate_14day'] = cases_per_state['confirmed_moving_avg_7day'] / cases_per_state['confirmed_moving_avg_7day'].shift(periods=14)
    cases_per_state['confirmed_trend'] = cases_per_state['confirmed_moving_avg_7day_rate_14day'].apply(get_trend)

    cases_per_state['deaths_per_day'] = cases_per_state['deaths'].diff(periods=1)
    cases_per_state['deaths_per_day'] = cases_per_state['deaths_per_day'].clip(lower=0)
    cases_per_state['deaths_moving_avg_7day'] = np.ceil(cases_per_state['deaths_per_day'].rolling(window=7).mean())
    cases_per_state['deaths_moving_avg_7day_rate_14day'] = cases_per_state['deaths_moving_avg_7day'] / cases_per_state['deaths_moving_avg_7day'].shift(periods=14)
    cases_per_state['deaths_trend'] = cases_per_state['deaths_moving_avg_7day_rate_14day'].apply(get_trend)

    if cases_is_empty:
        cases = cases_per_state
        cases_is_empty = False
    else:
        cases = pd.concat([cases, cases_per_state], axis=0, ignore_index=True)


cases_covid = cases
cases = None

Utiliza `astype('Int64')` nas colunas numéricas, como população, casos confirmados, e óbitos, garantindo que estejam no formato adequado, facilitando cálculos e análises subsequentes.  

In [155]:
cases_covid['population'] = cases_covid['population'].astype('Int64')
cases_covid['confirmed_per_day'] = cases_covid['confirmed_per_day'].astype('Int64')
cases_covid['confirmed_moving_avg_7day'] = cases_covid['confirmed_moving_avg_7day'].astype('Int64')
cases_covid['deaths_per_day'] = cases_covid['deaths_per_day'].astype('Int64')
cases_covid['deaths_moving_avg_7day'] = cases_covid['deaths_moving_avg_7day'].astype('Int64')

Ordena as colunas do Dataframe.

In [156]:
cases_covid = cases_covid[['date', 'country', 'state', 'population', 'confirmed', 'confirmed_per_day', 'confirmed_moving_avg_7day', 
    'confirmed_moving_avg_7day_rate_14day', 'confirmed_trend', 'deaths', 'deaths_per_day', 'deaths_moving_avg_7day', 
    'deaths_moving_avg_7day_rate_14day', 'deaths_trend', 'day', 'month']]

#### **2.1.3\. Carregamento**

Salva o DataFrame **`cases_covid`** em um arquivo CSV chamado `covid_cases_2021.csv`.  

In [157]:
cases_covid.to_csv('covid_cases_2021.csv', sep=',', index=False)

### 2.2. Vacinas Covid

 Os dados sobre **vacinação da COVID-19** são compilados pelo projeto Nosso Mundo em Dados (*Our World in Data* ou OWID) da universidade britânica de **Oxford** ([link](https://www.ox.ac.uk)). Os dados são **atualizados diariamente** deste janeiro de 2020 com uma **granularidade temporal de dias e geográfica de países**. O website do projeto pode ser acessado neste [link](https://ourworldindata.org) enquanto os dados, neste [link](https://covid.ourworldindata.org/data/owid-covid-data.csv). Abaixo estão descritos os dados derivados do seu processamento.

  - **date**: data de referência;
 - **country**: país;
 - **population**: população estimada;
 - **total**: número acumulado de doses administradas;
 - **one_shot**: número acumulado de pessoas com uma dose;
 - **one_shot_perc**: número acumulado relativo de pessoas com uma dose;
 - **two_shots**: número acumulado de pessoas com duas doses;
 - **two_shot_perc**: número acumulado relativo de pessoas com duas doses;
 - **three_shots**: número acumulado de pessoas com três doses;
 - **three_shot_perc**: número acumulado relativo de pessoas com três doses;
 - **day**: dia de referência;
 - **month**: mês de referência;

#### **2.2.1\. Extração**

Utiliza o método **`pd.read_csv`** para ler os dados de um arquivo CSV localizado na URL.  
Carregar os dados do arquivo CSV remoto, com especial atenção para a conversão da coluna de datas, que será útil para análise temporal.

In [158]:
vaccines = pd.read_csv('https://covid.ourworldindata.org/data/owid-covid-data.csv', sep=',', parse_dates=[3])

Salva o DataFrame **`vaccines`** em um arquivo CSV chamado `vaccines_covid.csv`.  
Redefine a variável **`vaccines`** como `None`, liberando memória e sinalizando que o DataFrame não está mais em uso.

In [159]:
vaccines.to_csv('vaccines_covid.csv', sep=',', index=False)
vaccines = None

Carrega o arquivo CSV `vaccines_covid.csv` em um DataFrame chamado **`vaccines_covid`**.  
O parâmetro `parse_dates=[3]` converte automaticamente a coluna 4 do arquivo em formato de data (`datetime`).  

In [170]:
vaccines_covid = pd.read_csv('vaccines_covid.csv', sep=',', parse_dates=[3])

#### **2.2.2\. Transformação**

Em **`vaccines_covid.query`**, seleciona os países de interesse.  

Depois seleciona as colunas de interesse.


In [171]:
vaccines_covid = vaccines_covid.query('location in ["Brazil", "Japan", "China", "Russia", "France", "Italy", "Ukraine", "Spain", "Canada", "Mexico", "Peru", "Colombia"]').reset_index(drop=True)
vaccines_covid = vaccines_covid[['location', 'population', 'total_vaccinations', 'people_vaccinated', 'people_fully_vaccinated', 'total_boosters', 'date']]

Filtra o DataFrame **`vaccines_covid`** para incluir apenas as linhas onde a coluna **`date`** está dentro do intervalo de 1º de janeiro de 2021 até 31 de dezembro de 2021.  
Limpa o índice do DataFrame após a filtragem, garantindo que a contagem de índices seja contínua e não haja valores de índice redundantes.


In [172]:
vaccines_covid = vaccines_covid[(vaccines_covid['date'] >= '2021-01-01') & (vaccines_covid['date'] <= '2021-12-31')].reset_index(drop=True) 

Renomeia as colunas no DataFrame

In [173]:
vaccines_covid = vaccines_covid.rename(columns={
    'location': 'country',
    'total_vaccinations': 'total',
    'people_vaccinated': 'one_shot',
    'people_fully_vaccinated': 'two_shots',
    'total_boosters': 'three_shots',
  })

Cria uma nova coluna chamada **`day`** e outra chamada **`month`** no DataFrame **`vaccines_covid`**.  
O atributo **`.dt.day`** extrai o número correspondente ao dia do mês (1 a 31).  
O atributo **`.dt.month`** extrai o número correspondente ao mês (1 para janeiro, 2 para fevereiro, etc.).  

In [174]:
vaccines_covid['day'] = pd.to_datetime(vaccines_covid['date']).dt.day
vaccines_covid['month'] = pd.to_datetime(vaccines_covid['date']).dt.month

Calcula as porcentagens de população vacinada com uma, duas e três doses da vacina, em relação à população total.  
As porcentagens são arredondadas para quatro casas decimais para facilitar a leitura e análise dos dados.


In [178]:
vaccines_covid['one_shot_perc'] = round(vaccines_covid['one_shot'] / vaccines_covid['population'], 4)
vaccines_covid['two_shots_perc'] = round(vaccines_covid['two_shots'] / vaccines_covid['population'], 4)
vaccines_covid['three_shots_perc'] = round(vaccines_covid['three_shots'] / vaccines_covid['population'], 4)

Faz o *type casting* das colunas, garantindo que estejam no formato adequado, facilitando cálculos e análises subsequentes.  

In [179]:
vaccines_covid['total'] = vaccines_covid['total'].astype('Int64')
vaccines_covid['one_shot'] = vaccines_covid['one_shot'].astype('Int64')
vaccines_covid['two_shots'] = vaccines_covid['two_shots'].astype('Int64')
vaccines_covid['three_shots'] = vaccines_covid['three_shots'].astype('Int64')

Ordena as colunas do Dataframe.

In [180]:
vaccines_covid = vaccines_covid[['date', 'country', 'population', 'total', 'one_shot', 'one_shot_perc', 'two_shots', 'two_shots_perc', 'three_shots', 'three_shots_perc', 'day', 'month']]

#### **2.2.3\. Carregamento**

Salva o DataFrame **`vaccines_covid`** em um arquivo CSV chamado `vaccines_covid_2021.csv`.  

In [181]:
vaccines_covid.to_csv('vaccines_covid_2021.csv', sep=',', index=False)